<a href="https://colab.research.google.com/github/RogerHeederer/LM_practice/blob/master/Pytorch_Bert_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/RogerHeederer/LM_practice.git

Cloning into 'LM_practice'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 5.8MB/s 
     |████████████████████████████████| 1.1MB 11.6MB/s 
     |████████████████████████████████| 890kB 25.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4050fc86e6acd56c9ed569f140299f0183528285ed704959600ad623b418c635
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [3]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [4]:
train_df = pd.read_csv('/content/nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('/content/nsmc/ratings_test.txt', sep='\t')

In [5]:
train_df[1:5]

,id,document,label
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


##데이터 정제하기

In [6]:
print("훈련 데이터 총 갯수: ", len(train_df))

print("중복 없는 유니크 데이터 갯수(훈련):", train_df['document'].nunique())

훈련 데이터 총 갯수:  150000
중복 없는 유니크 데이터 갯수(훈련): 146182


In [7]:
# 훈련 데이터 대상 중복 제거 부터 하자
train_df.drop_duplicates(subset=['document'], inplace=True)
print("중복 제거 후 훈련 데이터 총 개수 : ", len(train_df))

중복 제거 후 훈련 데이터 총 개수 :  146183


In [8]:
# NULL 값 존재 유무 체크 및 상세 정보 제공
print(train_df.isnull().values.any(),"\n")
print(train_df.isnull().sum(),"\n")
train_df.loc[train_df.document.isnull()]

True 

id          0
document    1
label       0
dtype: int64 



,id,document,label
25857,2172111,NaN,1


In [9]:
# NULL 값 제거
train_df.dropna(inplace=True)
print(train_df.isnull().values.any(),"\n")

False 



In [22]:
# 테스트 데이터 대상 중복 제거
print("테스트 데이터 총 갯수: ", len(test_df))
print("중복 없는 유니크 데이터 갯수(테스트):", test_df['document'].nunique())

테스트 데이터 총 갯수:  50000
중복 없는 유니크 데이터 갯수(테스트): 49157


In [23]:
test_df.drop_duplicates(subset=['document'], inplace=True)
print("중복 제거 후 훈련 데이터 총 개수 : ", len(test_df))

중복 제거 후 훈련 데이터 총 개수 :  49158


In [24]:
# NULL 값 존재 유무 체크 및 상세 정보 제공
print(test_df.isnull().values.any(),"\n")
print(test_df.isnull().sum(),"\n")
test_df.loc[test_df.document.isnull()]

True 

id          0
document    1
label       0
dtype: int64 



,id,document,label
5746,402110,NaN,1


In [25]:
# 테스트 데이터 NULL 값 제거
test_df.dropna(inplace=True)
print(test_df.isnull().values.any(),"\n")

False 



In [26]:
#전체 데이터의 50%만 샘플링하여 실험

train_df = train_df.sample(frac=0.5, random_state=999)
test_df = test_df.sample(frac=0.5, random_state=999)

## BERT 모델에 맞는 데이터 셋 만들어주기

In [27]:
class NsmcDataset(Dataset):
  '''네이버 영화 리뷰 감정 데이터 셋'''
  def __init__(self, df):
    self.df = df
  
  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    text = self.df.iloc[idx, 1]
    label = self.df.iloc[idx, 2]
    return text, label

In [28]:
#훈련에 사용될 데이터 로더 만들어주기
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=8, shuffle=True, num_workers=2)

In [29]:
nsmc_train_dataset.df.get

<bound method NDFrame.get of              id                                           document  label
36150   8518408  수백향도 그렇구 기황후는 좀낫네요 스토리 전개가 어느정도 현실에 맞아야지 이드라마 ...      0
128353  9636987  정말..반전어쩌구 하길래 잠잘시간 쪼개서 봤더만 .같은 영화.반전같은소리하고 있네....      0
51433   1598718               지금보니 좀 유치하네요,...당시로썬 좋았겠지만 그럭저럭 재밌다.      0
54714   8727363                             7세이하 유아용으로 참 좋은 영화입니다.      0
114323  9746504                            이 영화가 크리스토퍼 놀란 감독의 영화라닝      1
...         ...                                                ...    ...
108163  7978665                                코만도 북한새끼들 없애줬으면 좋겠다      1
91298   7006459  공포는 빼라 보다 잤다 ㅡ,.ㅡ 감독 옆에 있었으면 한대 후릴꺼 같은 이기분은 머지???      0
113835  5458165       16회 중 15회 남았다는게 슬퍼지려함. 박유천을 오래오래 보고 싶은 맘에...      1
26431   8902251                                         지연진짜좋아♡♡♡♡      1
112201  7224475                                        이걸 영화라고 만들어      0

[73091 rows x 3 columns]>

In [30]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')#WordPiece 토크나이져 사용
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased') #구글 pretrained 모델 사용
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [31]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 2
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/2] Iteration 500 -> Train Loss: 0.6973, Accuracy: 0.506
[Epoch 1/2] Iteration 1000 -> Train Loss: 0.6823, Accuracy: 0.560
[Epoch 1/2] Iteration 1500 -> Train Loss: 0.6398, Accuracy: 0.634
[Epoch 1/2] Iteration 2000 -> Train Loss: 0.5946, Accuracy: 0.690
[Epoch 1/2] Iteration 2500 -> Train Loss: 0.5655, Accuracy: 0.716
[Epoch 1/2] Iteration 3000 -> Train Loss: 0.5426, Accuracy: 0.734
[Epoch 1/2] Iteration 3500 -> Train Loss: 0.5235, Accuracy: 0.739
[Epoch 1/2] Iteration 4000 -> Train Loss: 0.5066, Accuracy: 0.755
[Epoch 1/2] Iteration 4500 -> Train Loss: 0.5008, Accuracy: 0.759
[Epoch 1/2] Iteration 5000 -> Train Loss: 0.4936, Accuracy: 0.765
[Epoch 1/2] Iteration 5500 -> Train Loss: 0.4885, Accuracy: 0.763
[Epoch 1/2] Iteration 6000 -> Train Loss: 0.4889, Accuracy: 0.768
[Epoch 1/2] Iteration 6500 -> Train Loss: 0.4763, Accuracy: 0.776
[Epoch 1/2] Iteration 7000 -> Train Loss: 0.4709, Accuracy: 0.777
[Epoch 1/2] Iteration 7500 -> Train Loss: 0.4765, Accuracy: 0.768
[Epoch 1/2]

In [34]:
#평가
model.eval()

# 평가에 사용될 데이터 로더 만들기
nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: ignored